# Session 5 – Orchestrátor pre viacerých agentov

Ukazuje jednoduchý dvojagentový proces (Výskumník -> Editor) pomocou Foundry Local.


### Vysvetlenie: Inštalácia závislostí
Inštaluje `foundry-local-sdk` a `openai` potrebné pre prístup k lokálnemu modelu a dokončenie chatu. Idempotentné.


# Scenár
Implementuje minimalistický orchestrátorový vzor s dvoma agentmi:
- **Agent výskumník** zhromažďuje stručné faktické body
- **Agent editor** prepisuje text pre jasnosť na úrovni vedenia

Ukazuje zdieľanú pamäť pre každého agenta, sekvenčné odovzdávanie medzivýstupov a jednoduchú funkciu pipeline. Rozšíriteľné na viac rolí (napr. Kritik, Overovateľ) alebo paralelné vetvy.

**Premenné prostredia:**
- `FOUNDRY_LOCAL_ALIAS` - Predvolený model na použitie (predvolené: phi-4-mini)
- `AGENT_MODEL_PRIMARY` - Primárny model agenta (prepíše ALIAS)
- `AGENT_MODEL_EDITOR` - Model agenta editor (predvolené: primárny)

**Referenčný SDK:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local

**Ako to funguje:**
1. **FoundryLocalManager** automaticky spustí službu Foundry Local
2. Stiahne a načíta určený model (alebo použije uloženú verziu)
3. Poskytuje kompatibilný endpoint OpenAI na interakciu
4. Každý agent môže používať iný model na špecializované úlohy
5. Zabudovaná logika opakovania elegantne rieši prechodné zlyhania

**Kľúčové vlastnosti:**
- ✅ Automatické vyhľadávanie a inicializácia služby
- ✅ Správa životného cyklu modelu (stiahnutie, cache, načítanie)
- ✅ Kompatibilita s OpenAI SDK pre známe API
- ✅ Podpora viacerých modelov na špecializáciu agentov
- ✅ Robustné spracovanie chýb s logikou opakovania
- ✅ Lokálna inferencia (bez potreby cloudového API)


In [16]:
# Install dependencies
!pip install -q foundry-local-sdk openai

### Vysvetlenie: Základné importy a typovanie
Zavádza dataclassy na ukladanie správ agentov a typové náznaky pre lepšiu prehľadnosť. Importuje Foundry Local manager + OpenAI klienta pre následné akcie agenta.


In [17]:
from dataclasses import dataclass, field
from typing import List
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI

### Vysvetlenie: Inicializácia modelu (SDK vzor)
Používa Foundry Local Python SDK na robustnú správu modelov:
- **FoundryLocalManager(alias)** - Automaticky spúšťa službu a načítava model podľa aliasu
- **get_model_info(alias)** - Prekladá alias na konkrétne ID modelu
- **manager.endpoint** - Poskytuje koncový bod služby pre klienta OpenAI
- **manager.api_key** - Poskytuje API kľúč (voliteľné pre lokálne použitie)
- Podporuje samostatné modely pre rôznych agentov (primárny vs editor)
- Zabudovaná logika opakovania s exponenciálnym oneskorením pre odolnosť
- Overenie pripojenia na zabezpečenie pripravenosti služby

**Hlavný SDK vzor:**
```python
manager = FoundryLocalManager(alias)
model_info = manager.get_model_info(alias)
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key)
```

**Správa životného cyklu:**
- Manažéri sú globálne uložené pre správne vyčistenie
- Každý agent môže používať iný model na špecializáciu
- Automatické vyhľadávanie služieb a spracovanie pripojenia
- Elegantné opakovanie s exponenciálnym oneskorením pri zlyhaniach

Toto zabezpečuje správnu inicializáciu pred začiatkom orchestrácie agentov.

**Referencie:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local


In [18]:
import time

# Environment configuration
PRIMARY_ALIAS = os.getenv('AGENT_MODEL_PRIMARY', os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini'))
EDITOR_ALIAS = os.getenv('AGENT_MODEL_EDITOR', PRIMARY_ALIAS)

# Store managers globally for proper lifecycle management
primary_manager = None
editor_manager = None

def init_model(alias: str, max_retries: int = 3):
    """Initialize Foundry Local manager with retry logic.
    
    Args:
        alias: Model alias to initialize
        max_retries: Number of retry attempts with exponential backoff
    
    Returns:
        Tuple of (manager, client, model_id, endpoint)
    """
    delay = 2.0
    last_err = None
    
    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Init] Starting Foundry Local for '{alias}' (attempt {attempt}/{max_retries})...")
            
            # Initialize manager - this starts the service and loads the model
            manager = FoundryLocalManager(alias)
            
            # Get model info to retrieve the actual model ID
            model_info = manager.get_model_info(alias)
            model_id = model_info.id
            
            # Create OpenAI client with manager's endpoint
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key or 'not-needed'
            )
            
            # Verify the connection with a simple test
            models = client.models.list()
            print(f"[OK] Initialized '{alias}' -> {model_id} at {manager.endpoint}")
            
            return manager, client, model_id, manager.endpoint
            
        except Exception as e:
            last_err = e
            if attempt < max_retries:
                print(f"[Retry {attempt}/{max_retries}] Failed to init '{alias}': {e}")
                print(f"[Retry] Waiting {delay:.1f}s before retry...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[ERROR] Failed to initialize '{alias}' after {max_retries} attempts")
    
    raise RuntimeError(f"Failed to initialize '{alias}' after {max_retries} attempts: {last_err}")

# Initialize primary model (for researcher)
print(f"\n{'='*80}")
print(f"Initializing Primary Model: {PRIMARY_ALIAS}")
print('='*80)
primary_manager, primary_client, PRIMARY_MODEL_ID, primary_endpoint = init_model(PRIMARY_ALIAS)

# Initialize editor model (may be same as primary)
if EDITOR_ALIAS != PRIMARY_ALIAS:
    print(f"\n{'='*80}")
    print(f"Initializing Editor Model: {EDITOR_ALIAS}")
    print('='*80)
    editor_manager, editor_client, EDITOR_MODEL_ID, editor_endpoint = init_model(EDITOR_ALIAS)
else:
    print(f"\n[Info] Editor using same model as primary")
    editor_manager = primary_manager
    editor_client, EDITOR_MODEL_ID = primary_client, PRIMARY_MODEL_ID
    editor_endpoint = primary_endpoint

print(f"\n{'='*80}")
print(f"[Configuration Summary]")
print('='*80)
print(f"  Primary Agent:")
print(f"    - Alias: {PRIMARY_ALIAS}")
print(f"    - Model: {PRIMARY_MODEL_ID}")
print(f"    - Endpoint: {primary_endpoint}")
print(f"\n  Editor Agent:")
print(f"    - Alias: {EDITOR_ALIAS}")
print(f"    - Model: {EDITOR_MODEL_ID}")
print(f"    - Endpoint: {editor_endpoint}")
print('='*80)



Initializing Primary Model: phi-4-mini
[Init] Starting Foundry Local for 'phi-4-mini' (attempt 1/3)...
[OK] Initialized 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4 at http://127.0.0.1:59959/v1

Initializing Editor Model: gpt-oss-20b
[Init] Starting Foundry Local for 'gpt-oss-20b' (attempt 1/3)...
[OK] Initialized 'gpt-oss-20b' -> gpt-oss-20b-cuda-gpu:1 at http://127.0.0.1:59959/v1

[Configuration Summary]
  Primary Agent:
    - Alias: phi-4-mini
    - Model: Phi-4-mini-instruct-cuda-gpu:4
    - Endpoint: http://127.0.0.1:59959/v1

  Editor Agent:
    - Alias: gpt-oss-20b
    - Model: gpt-oss-20b-cuda-gpu:1
    - Endpoint: http://127.0.0.1:59959/v1


### Vysvetlenie: Triedy Agent & Memory
Definuje ľahkú triedu `AgentMsg` pre záznamy pamäte a triedu `Agent`, ktorá zahŕňa:
- **Úloha systému** - Persona agenta a jeho inštrukcie
- **História správ** - Udržiava kontext konverzácie
- **Metóda act()** - Vykonáva akcie s vhodným spracovaním chýb

Agent môže používať rôzne modely (primárny vs editor) a udržiava izolovaný kontext pre každého agenta. Tento vzor umožňuje:
- Zachovanie pamäte medzi akciami
- Flexibilné priradenie modelu pre každého agenta
- Izoláciu chýb a obnovu
- Jednoduché prepojenie a orchestráciu


In [19]:
@dataclass
class AgentMsg:
    role: str
    content: str

@dataclass
class Agent:
    name: str
    system: str
    client: OpenAI = None  # Allow per-agent client assignment
    model_id: str = None   # Allow per-agent model
    memory: List[AgentMsg] = field(default_factory=list)

    def _history(self):
        """Return chat history in OpenAI messages format including system + memory."""
        msgs = [{'role': 'system', 'content': self.system}]
        for m in self.memory[-6:]:  # Keep last 6 messages to avoid context overflow
            msgs.append({'role': m.role, 'content': m.content})
        return msgs

    def act(self, prompt: str, temperature: float = 0.4, max_tokens: int = 300):
        """Send a prompt, store user + assistant messages in memory, and return assistant text.
        
        Args:
            prompt: User input/task for the agent
            temperature: Sampling temperature (0.0-1.0)
            max_tokens: Maximum tokens to generate
        
        Returns:
            Assistant response text
        """
        # Use agent-specific client/model or fall back to primary
        client_to_use = self.client or primary_client
        model_to_use = self.model_id or PRIMARY_MODEL_ID
        
        self.memory.append(AgentMsg('user', prompt))
        
        try:
            # Build messages including system prompt and history
            messages = self._history() + [{'role': 'user', 'content': prompt}]
            
            resp = client_to_use.chat.completions.create(
                model=model_to_use,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            
            # Validate response
            if not resp.choices:
                raise RuntimeError("No completion choices returned")
            
            out = resp.choices[0].message.content or ""
            
            if not out:
                raise RuntimeError("Empty response content")
            
        except Exception as e:
            out = f"[ERROR:{self.name}] {type(e).__name__}: {str(e)}"
            print(f"[Agent Error] {self.name}: {type(e).__name__}: {str(e)}")
        
        self.memory.append(AgentMsg('assistant', out))
        return out

print("[INFO] Agent classes initialized with Foundry SDK support")
print(f"[INFO] Using OpenAI SDK version: {OpenAI.__module__}")


[INFO] Agent classes initialized with Foundry SDK support
[INFO] Using OpenAI SDK version: openai


### Vysvetlenie: Orchestrated Pipeline
Vytvára dvoch špecializovaných agentov:
- **Výskumník**: Používa primárny model, zhromažďuje faktické informácie
- **Editor**: Môže používať samostatný model (ak je nakonfigurovaný), upravuje a prepisuje

Funkcia `pipeline`:
1. Výskumník zhromažďuje surové informácie
2. Editor ich upravuje do výstupu pripraveného pre vedenie
3. Vracia medzivýsledky aj konečné výsledky

Tento vzor umožňuje:
- Špecializáciu modelov (rôzne modely pre rôzne úlohy)
- Zlepšenie kvality prostredníctvom viacstupňového spracovania
- Sledovateľnosť transformácie informácií
- Jednoduché rozšírenie o ďalších agentov alebo paralelné spracovanie


In [ ]:
# Create specialized agents with optional model assignment
researcher = Agent(
    name='Researcher',
    system='You collect concise factual bullet points.',
    client=primary_client,
    model_id=PRIMARY_MODEL_ID
)

editor = Agent(
    name='Editor',
    system='You rewrite content for clarity and an executive, action-focused tone.',
    client=editor_client,
    model_id=EDITOR_MODEL_ID
)

def pipeline(q: str, verbose: bool = True):
    """Execute multi-agent pipeline: Researcher -> Editor.
    
    Args:
        q: User question/task
        verbose: Print intermediate outputs
    
    Returns:
        Dictionary with research, final outputs, and metadata
    """
    if verbose:
        print(f"[Pipeline] Question: {q}\n")
    
    # Stage 1: Research
    if verbose:
        print("[Stage 1: Research]")
    research = researcher.act(q)
    if verbose:
        print(f"Output: {research[:200]}...\n")
    
    # Stage 2: Editorial refinement
    if verbose:
        print("[Stage 2: Editorial Refinement]")
    rewrite = editor.act(
        f"Rewrite professionally with a 1-sentence executive summary first. "
        f"Improve clarity, keep bullet structure if present. Source:\n{research}"
    )
    if verbose:
        print(f"Output: {rewrite[:200]}...\n")
    
    return {
        'question': q,
        'research': research,
        'final': rewrite,
        'models': {
            'researcher': PRIMARY_MODEL_ID,
            'editor': EDITOR_MODEL_ID
        }
    }

# Execute sample pipeline
print("="*80)
result = pipeline('Explain why edge AI matters for compliance and latency.')
print("="*80)
print("\n[FINAL OUTPUT]")
print(result['final'])
print("\n[METADATA]")
print(f"Models used: {result['models']}")
result

[Pipeline] Question: Explain why edge AI matters for compliance and latency.

[Stage 1: Research]
Output: - **Data Sovereignty**: Edge AI allows data to be processed locally, which can help organizations comply with regional data protection regulations by keeping sensitive information within the borders o...

[Stage 2: Editorial Refinement]


### Vysvetlenie: Spustenie a výsledky pipeline
Spúšťa multi-agentnú pipeline na otázku s témou súladu a latencie, aby demonštrovala:
- Viacstupňovú transformáciu informácií
- Špecializáciu a spoluprácu agentov
- Zlepšenie kvality výstupu prostredníctvom zdokonaľovania
- Sledovateľnosť (zachovanie medzivýstupov aj konečných výstupov)

**Štruktúra výsledku:**
- `question` - Pôvodná otázka používateľa
- `research` - Surový výskumný výstup (faktické body)
- `final` - Zdokonalené výkonné zhrnutie
- `models` - Ktoré modely boli použité v každej fáze

**Nápady na rozšírenie:**
1. Pridať agenta Kritika na kontrolu kvality
2. Implementovať paralelných výskumných agentov pre rôzne aspekty
3. Pridať agenta Overovateľa na kontrolu faktov
4. Použiť rôzne modely pre rôzne úrovne zložitosti
5. Implementovať spätnoväzobné cykly na iteratívne zlepšovanie


### Pokročilé: Konfigurácia vlastného agenta

Skúste prispôsobiť správanie agenta úpravou premenných prostredia pred spustením inicializačnej bunky:

**Dostupné modely:**
- Použite `foundry model ls` v termináli na zobrazenie všetkých dostupných modelov
- Príklady: phi-4-mini, phi-3.5-mini, qwen2.5-7b, llama-3.2-3b, atď.


In [ ]:
# Example: Use different models for different agents
# Uncomment and modify as needed:

# import os
# os.environ['AGENT_MODEL_PRIMARY'] = 'phi-4-mini'      # Fast, good for research
# os.environ['AGENT_MODEL_EDITOR'] = 'qwen2.5-7b'       # Higher quality for editing

# Then restart the kernel and re-run all cells

# Test with different questions
test_questions = [
    "What are 3 key benefits of using small language models?",
    "How does RAG improve AI accuracy?",
    "Why is local inference important for privacy?"
]

print("Testing pipeline with multiple questions:\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {q}")
    print('='*80)
    r = pipeline(q, verbose=False)
    print(f"\n[FINAL]: {r['final'][:300]}...")
    print(f"[Models]: Researcher={r['models']['researcher']}, Editor={r['models']['editor']}")


Testing pipeline with multiple questions:


Question 1: What are 3 key benefits of using small language models?

[FINAL]: <|channel|>analysis<|message|>The user wants a rewrite of the entire block of text. The rewrite should be professional, include a one-sentence executive summary first, improve clarity, keep bullet structure if present. The user has provided a large amount of text. The user wants a rewrite of that te...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 2: How does RAG improve AI accuracy?

[FINAL]: <|channel|>final<|message|>**RAG (Retrieval‑Augmented Generation) empowers AI to produce highly accurate, contextually relevant responses by combining a retrieval system with a large language model (LLM).**<|return|>...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 3: Why is local inference important for privacy?

[FINAL]: <|channel|>final<|message|>**Local inference—processing data d


---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby AI prekladu [Co-op Translator](https://github.com/Azure/co-op-translator). Hoci sa snažíme o presnosť, prosím, berte na vedomie, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho rodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nenesieme zodpovednosť za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
